# Sampling to equal read depth from pairs

In [ ]:
# Import the packages we will use
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py
import bioframe
from matplotlib.gridspec import GridSpec
from matplotlib.gridspec import GridSpecFromSubplotSpec
import matplotlib.colors as colors
import cooler
import seaborn as sns
from matplotlib.colors import ListedColormap
import os
import glob
import pairtools
import random

In [36]:
#For many libraries
inDir = "/nl/umw_job_dekker/users/as38w/JZ2022_git/pairs/"
outDir = "/nl/umw_job_dekker/users/as38w/JZ2022_git/coolers/"

conditions = [
    'DMSO-ND',
    'DMSO-4OHT'
        
]

long_names = {
    'DMSO-ND':'JZ-MEF-AsiS1-ND-dmso6h-HiCD2R1',
    'DMSO-4OHT':'JZ-MEF-AsiS1-4OHT-dmso6h-HiCD2R1'
}

pairsPath = {}
for cond in conditions[0:2]:
    pairsPath[cond] = '{}/{}__mm10.mm10.nodups.pairs.gz'.format(inDir, long_names[cond])
    

In [37]:
pairsPath

{'DMSO-ND': '/nl/umw_job_dekker/users/as38w/JZ2022_git/pairs//JZ-MEF-AsiS1-ND-dmso6h-HiCD2R1__mm10.mm10.nodups.pairs.gz',
 'DMSO-4OHT': '/nl/umw_job_dekker/users/as38w/JZ2022_git/pairs//JZ-MEF-AsiS1-4OHT-dmso6h-HiCD2R1__mm10.mm10.nodups.pairs.gz'}

In [38]:
noheaderPath = {}
for cond in conditions:
    noheaderPath[cond] = '{}/{}.mm10.nodups.noheader.pairs'.format(outDir, long_names[cond])
    

In [39]:
#Remove header and unzip noheader file
for cond in conditions:
    infile = pairsPath[cond]
    noheaderfile = noheaderPath[cond]
    !bsub -q short -W 04:00 -n 4 -R "span[hosts=1]" -R rusage[mem=12000] -R "select[rh=8]"\
        "zgrep -o '^[^#]*' $infile > $noheaderfile"
        

INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543594> is submitted to queue <short>.
INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543595> is submitted to queue <short>.


In [40]:
sampledPaths = {}

for cond in conditions:
    sampledPaths[cond] = '{}/{}.mm10.nodups.noheader_sampled_64509240M.pairs'.format(outDir, long_names[cond])
 

In [41]:
#shuffle
for cond in conditions:
    noheaderfile = noheaderPath[cond]
    shuffile = sampledPaths[cond]
    !bsub -q short -W 04:00 -n 4 -R "span[hosts=1]" -R rusage[mem=24000] -R "select[rh=8]"\
        "shuf -n 64509240 -o $shuffile $noheaderfile"
        

INFO: Total memory requested is 96000 MB (4 cores x 24000 MB)
Job <1543596> is submitted to queue <short>.
INFO: Total memory requested is 96000 MB (4 cores x 24000 MB)
Job <1543597> is submitted to queue <short>.


In [47]:
#add header before sorting
headerPaths = {}

for cond in conditions:
    headerPaths[cond] = '{}/{}.mm10.nodups.header_sampled_65M.pairs'.format(outDir, long_names[cond])
    

In [48]:
#add header to sampled files ... #IMPORTANT: need the chromsizes (1-22, X, Y, M) and columns line.  I made this into one file:
headerfile = '/nl/umw_job_dekker/users/as38w/JZ-MEF-2020/sampled_2022/pairsheader_mm10'
for cond in conditions:
    in_fname = sampledPaths[cond]
    tmp_fname = headerPaths[cond]
    !bsub -q short -W 01:00 -n 4 -R "span[hosts=1]" -R rusage[mem=12000] -R "select[rh=8]"\
        "cat $headerfile $in_fname > $tmp_fname"


INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543602> is submitted to queue <short>.
INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543603> is submitted to queue <short>.


In [49]:
sortedPaths = {}

for cond in conditions:
    sortedPaths[cond] = '{}/{}.mm10.nodups.sorted.sampled_65M.pairs'.format(outDir, long_names[cond])
    

In [50]:
#sort sampled files plus header
for cond in conditions:
    in_fname = headerPaths[cond]
    out_fname = sortedPaths[cond]
    !bsub -q short -W 04:00 -n 2 -R span[hosts=1] -R select[ib] -R "select[rh=8]" -R rusage[mem=12000] "source activate coolerenv_manual_master; pairtools sort -o $out_fname $in_fname"


Job <1543604> is submitted to queue <short>.
Job <1543605> is submitted to queue <short>.


In [51]:
#rezip using bgzip on sampled files
for cond in conditions:
    in_fname = sortedPaths[cond]
    !bsub -q short -W 04:00 -n 4 -R "span[hosts=1]" -R rusage[mem=24000] -R "select[rh=8]"\
        "bgzip $in_fname"
        

INFO: Total memory requested is 96000 MB (4 cores x 24000 MB)
Job <1543606> is submitted to queue <short>.
INFO: Total memory requested is 96000 MB (4 cores x 24000 MB)
Job <1543607> is submitted to queue <short>.


In [53]:
#For many libraries
inDir = "/nl/umw_job_dekker/users/as38w/JZ2022_git/pairs/"
outDir = "/nl/umw_job_dekker/users/as38w/JZ2022_git/coolers/"

newconditions = [
    'DMSO-ND',
    'DMSO-4OHT'
        
]

newlong_names = {
    'DMSO-ND':'JZ-MEF-AsiS1-ND-dmso6h-HiCD2R1',
    'DMSO-4OHT':'JZ-MEF-AsiS1-4OHT-dmso6h-HiCD2R1'
}

newPairsPath = {}
for cond in newconditions:
    newPairsPath[cond] = '{}/{}.mm10.nodups.sorted.sampled_65M.pairs.gz'.format(outDir, newlong_names[cond])
    

In [54]:
#index the pairs files after sorting
for cond in newconditions:
    in_fname = newPairsPath[cond]
    !bsub -q short -W 04:00 -n 4 -R "span[hosts=1]" -R rusage[mem=24000] -R "select[rh=8]"\
        "pairix -p pairs $in_fname"
        

INFO: Total memory requested is 96000 MB (4 cores x 24000 MB)
Job <1543608> is submitted to queue <short>.
INFO: Total memory requested is 96000 MB (4 cores x 24000 MB)
Job <1543609> is submitted to queue <short>.


In [55]:
#stats
for cond in newconditions:
    in_fname = newPairsPath[cond]
    out_fname = '{}/{}.mm10.nodups.sorted.sampled_65M.pairs.stats.tsv'.format(outDir, newlong_names[cond])
    !bsub -q short -W 03:00 -n 4 -R "span[hosts=1]" -R rusage[mem=12000] -R "select[rh=8]"\
        "pairtools stats -o $out_fname $in_fname"
       

INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543610> is submitted to queue <short>.
INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543611> is submitted to queue <short>.


In [56]:
chromsizes="/nl/umw_job_dekker/cshare/reference/sorted_chromsizes/mm10.reduced.chrom.sizes"
binfile="/nl/umw_job_dekker/users/as38w/JZ2022_git/coolers/mm10_1000_bins"

!bsub -q short -W 03:00 -n 4 -R "span[hosts=1]" -R "select[rh=8]" -R rusage[mem=12000] "source activate coolerenv_manual_master; cooler makebins -o $binfile $chromsizes 1000"


INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543612> is submitted to queue <short>.


In [57]:
#make 1kb sampled coolers
binfile = '/nl/umw_job_dekker/users/as38w/JZ2022_git/coolers/mm10_1000_bins'

for cond in newconditions:
    in_fname = newPairsPath[cond]
    coolfile = '{}/{}.mm10.nodups.65M.1000.cool'.format(outDir, newlong_names[cond])
    !bsub -q short -W 03:00 -n 4 -R "span[hosts=1]" -R "select[rh=8]" -R rusage[mem=12000] "source activate coolerenv_manual_master; cooler cload pairs --assembly mm10 -c1 2 -p1 3 -c2 4 -p2 5 $binfile $in_fname $coolfile"
        

INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543614> is submitted to queue <short>.
INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543615> is submitted to queue <short>.


In [59]:
#balance single resolution coolers
for cond in newconditions:
    coolfile = '{}/{}.mm10.nodups.65M.1000.cool'.format(outDir, newlong_names[cond])
    !bsub -q short -W 03:00 -n 4 -R "span[hosts=1]" -R "select[rh=8]" -R rusage[mem=12000] "source activate coolerenv_manual_master; cooler balance -p 8 $coolfile"
                            

INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543641> is submitted to queue <short>.
INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543642> is submitted to queue <short>.


In [60]:
#zoomify and balance 1kb coolers into mcools
res_str = '10000000,5000000,2500000,1000000,500000,250000,100000,50000,25000,10000,5000,2000,1000'

#sampled files
for cond in newconditions:
    coolfile = '{}/{}.mm10.nodups.65M.1000.cool'.format(outDir, newlong_names[cond])
    mcoolfile = '{}/{}.mm10.nodups.65M.1000.mcool'.format(outDir, newlong_names[cond])
    !bsub -q short -W 03:00 -n 4 -R "span[hosts=1]" -R "select[rh=8]" -R rusage[mem=12000] "source activate coolerenv_updated_master; cooler zoomify -p 8 --balance --resolutions $res_str $coolfile -o $mcoolfile"


INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543643> is submitted to queue <short>.
INFO: Total memory requested is 48000 MB (4 cores x 12000 MB)
Job <1543644> is submitted to queue <short>.
